This is the file that orders the scraped data from the wiki and inserts it into SQL 

Imports and such

In [1]:
import pandas as pd
import psycopg2
from pathlib import Path
from tqdm.notebook import trange, tqdm
from hf import psyCop, clean_text, replaceWithNull, clean_textLigt
import numpy as np
from os.path import exists
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import psycopg2
from pathlib import Path
#from tqdm.notebook import trange, tqdm
from hf import  clean_text, replaceWithNull
import numpy as np
from cleantext import clean
from os.path import exists

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Timer

In [2]:
startTid = time.time()

In [3]:
file = Path("../Data/wiki.csv")

# Code for reading the csv file and separating.
data = []
i = 0
breakpoint = 100
chunksize = 100

In [4]:
with open (file, 'r', encoding='utf-8') as reader: 
    reader = pd.read_csv(reader, chunksize=chunksize,quotechar='"', delimiter=',')
    for chunk in reader:
        replaceWithNull(chunk)
        for column in chunk.columns:
            chunk[column] = chunk[column].astype(str)
            chunk[column] = chunk[column].str.replace("\n", "")
            chunk[column] = chunk[column].replace("nan", "null")
        data.append(chunk)
        i += 1
        if i >= breakpoint:
            break           
reader.close()
data = pd.concat(data)

# Converting to a pandas dataframe. 
df = pd.DataFrame(data)   
df.columns = ['Title' ,'Date published','Article contentNone']

In [5]:
print (df.shape)
print (df.iloc[:,1])

(1212, 3)
0          sunday   DATE   NUM  
1        saturday   DATE   NUM  
2          monday   DATE   NUM  
3          monday   DATE   NUM  
4       wednesday   DATE   NUM  
                  ...           
1207       friday   DATE   NUM  
1208       friday   DATE   NUM  
1209      tuesday   DATE   NUM  
1210      tuesday   DATE   NUM  
1211      tuesday   DATE   NUM  
Name: Date published, Length: 1212, dtype: object


In [6]:
#Droper Tables hvis de eksitere 
drop_tables = (""" DROP TABLE IF EXISTS pap_article""") 

#Tables til Wikipedia
create_politics_table = ("""
    CREATE TABLE IF NOT EXISTS pap_article(
    id  SERIAL PRIMARY KEY,
    title VARCHAR,
    published VARCHAR,
    content VARCHAR     
    )
    """
)

In [7]:
conn = None
try:
    conn = psycopg2.connect(#database="public",
                        port = "5432",
                        host="localhost", 
                        user="postgres", 
                        password="1234")
except psycopg2.DatabaseError as e: 
        print (f'Error {e}')
        sys.exit(1)

# Open a cursor to perform database operations
cur = conn.cursor()
cur.execute(drop_tables)
cur.execute(create_politics_table)

with open('../Data/wiki.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "pap_article", columns=('title', "published", "content"), sep=",", null="")

# Make the changes to the database persistent
conn.commit()

# Close communication with the database
cur.close()
conn.close()